In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy

from importlib import reload
from matplotlib import rc
from neurobiases import (TriangularModel,
                         EMSolver,
                         em_utils,
                         solver_utils)
%matplotlib inline

In [ ]:
K = 1

In [ ]:
# Create triangular model and draw data
tm = TriangularModel(
    model='linear',
    parameter_design='direct_response',
    M=10,
    N=10,
    K=K,
    corr_cluster=0.25,
    corr_back=0.1,
    coupling_distribution='gaussian',
    coupling_sparsity=0.5,
    coupling_loc=0,
    coupling_scale=0.25,
    coupling_rng=2332,
    tuning_distribution='gaussian',
    tuning_sparsity=0.5,
    tuning_loc=0,
    tuning_scale=0.25,
    tuning_rng=23456542,
    stim_distribution='uniform')
X, Y, y = tm.generate_samples(n_samples=500, rng=2332)

In [ ]:
true = EMSolver(
    X, Y, y, K=K,
    solver='scipy_lbfgs',
    max_iter=500,
    tol=1e-8,
    c_coupling=0,
    c_tuning=0,
    penalize_B=False,
    initialization='random')
true.set_params(a=tm.a, b=tm.b, B=tm.B,
                Psi_tr=true.Psi_to_Psi_tr(tm.Psi), L=tm.L)
true_grad = true.mll_grad(wrt_Psi=True)
true_grad /= np.linalg.norm(true_grad)

In [ ]:
n_reps = 30
non_sparse_w_init_dds = np.zeros(n_reps)
non_sparse_w_true_dds = np.zeros(n_reps)
sparse_w_init_dds = np.zeros(n_reps)
sparse_w_true_dds = np.zeros(n_reps)
random_dds = np.zeros(1000)

In [ ]:
for idx in range(1000):
    random_vec = np.random.normal(loc=0, scale=1, size=true_grad.shape)
    random_vec /= np.linalg.norm(random_vec)
    random_dds[idx] = random_vec @ true_grad

In [ ]:
for rep in range(n_reps):
    # Run sparse solver
    sparse = EMSolver(
        X, Y, y, K=K,
        solver='scipy_lbfgs',
        max_iter=500,
        tol=1e-8,
        a_mask=tm.a.ravel() != 0,
        b_mask=tm.b.ravel() != 0,
        c_coupling=0,
        c_tuning=0,
        penalize_B=False,
        initialization='random')
    sparse_grad = sparse.mll_grad(wrt_Psi=True)
    sparse_grad /= np.linalg.norm(sparse_grad)
    sparse_init = sparse.get_params(return_Psi=True)
    sparse.fit_em(refit=False)
    init_to_fit = sparse.get_params(return_Psi=True) - sparse_init
    init_to_fit /= np.linalg.norm(init_to_fit)
    sparse_w_init_dds[rep] = sparse_grad @ init_to_fit
    
    true_to_fit = sparse.get_params(return_Psi=True)[:20] - true.get_params(return_Psi=True)[:20]
    true_to_fit /= np.linalg.norm(true_to_fit)
    sparse_w_true_dds[rep] = true_grad[:20] @ true_to_fit
    
    # Run non-sparse solver
    non_sparse = EMSolver(
        X, Y, y, K=K,
        solver='scipy_lbfgs',
        max_iter=500,
        tol=1e-8,
        c_coupling=0,
        c_tuning=0,
        penalize_B=False,
        initialization='random')
    non_sparse_grad = non_sparse.mll_grad(wrt_Psi=True)
    non_sparse_grad /= np.linalg.norm(non_sparse_grad)
    non_sparse_init = non_sparse.get_params(return_Psi=True)
    non_sparse.fit_em(refit=False)
    init_to_fit = non_sparse.get_params(return_Psi=True) - non_sparse_init
    init_to_fit /= np.linalg.norm(init_to_fit)
    non_sparse_w_init_dds[rep] = non_sparse_grad @ init_to_fit
    
    true_to_fit = non_sparse.get_params(return_Psi=True) - true.get_params(return_Psi=True) 
    true_to_fit /= np.linalg.norm(true_to_fit)
    non_sparse_w_true_dds[rep] = true_grad @ true_to_fit

In [ ]:
plt.hist(sparse_w_true_dds)

In [ ]:
plt.hist(non_sparse_w_init_dds)
plt.hist(sparse_w_init_dds)

In [ ]:
plt.hist(sparse_w_true_dds)
plt.hist(non_sparse_w_true_dds)
plt.hist(non_sparse_w_init_dds)
plt.hist(sparse_w_init_dds)